In [ ]:
# read command line arguments
import os
import argparse
parser = argparse.ArgumentParser(description='Experiment')
parser.add_argument('--ns', type=int, default=4, help='number of virtual points')

args = parser.parse_args(os.environ['NB_ARGS'].split())
# args = parser.parse_args("")
ns = args.ns
# ns = 128 ns should be 2^n
print("ns = ", ns)

In [ ]:
import cpuinfo

In [ ]:
from jax import config
import jax.numpy as jnp
import jax.random as jr
import optax as ox
from jaxtyping import install_import_hook
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import qmc
from numpy.random import Generator, PCG64

import numpy as np
import torch
import scipy
import copy
import timeit
import os

from constrained_gp import *

config.update("jax_enable_x64", True)

with install_import_hook("gpjax", "beartype.beartype"):
    import gpjax as gpx

key = jr.key(123)

cols = mpl.rcParams["axes.prop_cycle"].by_key()["color"]

print_title = False
save_fig = True
save_samples = True
file_name = "3d_convection_diffusion_"+str(ns)
# create dir file_name 
if not os.path.exists(file_name):
    os.makedirs(file_name)
info_strings = []
fig_size = (8,6)#(1.5, 1.5)

In [ ]:
vels = jnp.load("vels.npy")
ts = jnp.load("ts.npy")
xs = jnp.load("xs.npy")
ys = jnp.load("ys.npy")

In [ ]:
vmin=-0.05
vmax=1.1

In [ ]:
m,n,q = ys.shape
x = vels#jnp.linspace(0, 1, m)
y = ts#jnp.linspace(0, 1, n)
z = xs#jnp.linspace(0, 1, q)
# create a meshgrid
X, Y, Z = jnp.meshgrid(x, y, z, indexing='ij')
# create a 3D scatter plot
fig = plt.figure()#figsize=fig_size
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X, Y, Z, c=ys, cmap='viridis', marker='o', vmin=vmin, vmax=vmax)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('T')
plt.show()

In [ ]:
np.random.seed(0)
no_of_samples = 64#32#64#32#64#32#64#128
vel_indices = np.random.choice(ys.shape[0], no_of_samples)
# print(vel_indices)
t_indices = np.random.choice(ys.shape[1], no_of_samples)
# print(t_indices)
x_indices = np.random.choice(ys.shape[2], no_of_samples)
# print(x_indices)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(vels[vel_indices], ts[t_indices], xs[x_indices], c=ys[vel_indices, t_indices, x_indices], vmin=vmin, vmax=vmax)
ax.set_xlabel("V")
ax.set_ylabel("T")
ax.set_zlabel("X")


In [ ]:
# training data
t = jnp.vstack([vels[vel_indices], ts[t_indices], xs[x_indices]]).T
f0t = ys[vel_indices, t_indices, x_indices].reshape(-1, 1)

# test data
u = jnp.vstack([X.flatten(), Y.flatten(), Z.flatten()]).T
f0u_exact = ys.flatten().reshape(-1,1)


In [ ]:
# d = 16
noise = 1e-4#1e-6#0.3#0.01#-3#1e-3#5e-1#5
xlim = (vels[0], vels[-1])
ylim = (ts[0], ts[-1])
zlim = (xs[0], xs[-1])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(u[:,0], u[:,1], u[:,2], c=f0u_exact, vmin=vmin, vmax=vmax)
ax.set_xlabel("V")
ax.set_ylabel("T")
ax.set_zlabel("X")
ax.set_title("Ground truth")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(u[:,0], u[:,1], u[:,2], c=f0u_exact, vmin=vmin, vmax=vmax)

ax.scatter(t[:,0], t[:,1], t[:,2], c='gray', label='Data Points')

ax.set_xlabel("V")
ax.set_ylabel("T")
ax.set_zlabel("X")
ax.set_title("Ground truth")

## Build unconstrained GP model

In [ ]:
# data
D = gpx.Dataset(X=t, y=f0t)
# prior
kernel = gpx.kernels.RBF(lengthscale=1.0, variance=10)#1.3
# meanf = gpx.mean_functions.Constant(0.0)
meanf = gpx.mean_functions.Constant(0.0)
prior = gpx.gps.Prior(mean_function=meanf, kernel=kernel)

# posterior
likelihood = gpx.likelihoods.Gaussian(num_datapoints=D.n, obs_stddev=noise) #1e-3
posterior = prior * likelihood

opt_posterior, history = gpx.fit(
    model=posterior,
    objective=lambda p, d: -gpx.objectives.conjugate_mll(p, d),
    optim=ox.sgd(0.0001),
    train_data=D,
    num_iters=20000
)

print("optimized lengthscale:", opt_posterior.prior.kernel.lengthscale.value)
print("optimized variance:", opt_posterior.prior.kernel.variance.value)
print("optimized mean:", opt_posterior.prior.mean_function.constant.value)
print("optimized noise std:", opt_posterior.likelihood.obs_stddev.value)

# shorter names for optimized parameters
l = opt_posterior.prior.kernel.lengthscale.value#*0.5
opt_posterior.prior.kernel.lengthscale.value = opt_posterior.prior.kernel.lengthscale.value#*0.5
sigma = jnp.sqrt(opt_posterior.prior.kernel.variance.value)
sigma_square = opt_posterior.prior.kernel.variance.value
fun_noise_var = opt_posterior.likelihood.obs_stddev.value**2
fun_noise_matrix = jnp.eye(len(t)) * (fun_noise_var) +1e-1
fun_noise_matrix = np.asarray(fun_noise_matrix)

In [ ]:
unconstrained_gp = opt_posterior.predict(u, train_data=D)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
im = ax.scatter(u[:,0], u[:,1], u[:,2], c=unconstrained_gp.mean, vmin=vmin, vmax=vmax)
ax.scatter(t[:,0], t[:,1], t[:,2], c='gray', label='Data Points')
ax.set_xlabel("V")
ax.set_ylabel("T")
ax.set_zlabel("X")
# plt.colorbar(im, ax=ax)

info_string = "%.2e %.2e" % (mean_squared_error(f0u_exact, unconstrained_gp.mean), 4*unconstrained_gp.stddev().mean())
info_strings.append(info_string)
print(info_string)

In [ ]:
fig, ax = plot_3d_slices(unconstrained_gp.mean, (u[:,0], u[:,1], u[:,2]), z_values=[1.0], vmin=vmin, vmax=vmax,xlim=xlim,ylim=ylim,zlim=zlim, figsize=fig_size)

ax.set_xlabel("Velocity")
ax.set_ylabel("Time")
ax.set_zlabel("Space")
plt.tight_layout()
if save_fig:
    plt.savefig(file_name + "/unconstrained_mean.pdf", bbox_inches="tight")

std_colorbar_min = 4*unconstrained_gp.stddev().min()
std_colorbar_max = 4*unconstrained_gp.stddev().max()
fig, ax = plot_3d_slices(4*unconstrained_gp.stddev(), (u[:,0], u[:,1], u[:,2]), z_values=[1.0], vmin=std_colorbar_min, vmax=std_colorbar_max,xlim=xlim,ylim=ylim,zlim=zlim, figsize=fig_size)

ax.set_xlabel("Velocity")
ax.set_ylabel("Time")
ax.set_zlabel("Space")
plt.tight_layout()

if save_fig:
    plt.savefig(file_name + "/unconstrained_ci.pdf", bbox_inches="tight")

In [ ]:
fig, ax = plot_isosurfaces(
    coords=u,                     
    values=unconstrained_gp.mean,
    isovalues=np.linspace(0.1, 0.9, 5),
    alphas=0.5,
    show_data_points=False,
    data_points=t,
    figsize=(5, 5),  # Increase width to give more space
    grid_size=10
)
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_zlim(zlim)

plt.show()


## Virtual points

In [ ]:
# ns = 32
s_dim = ns

# # LHC
# s_dim = 4
# rng = Generator(PCG64(2025))
# lh_sampler = qmc.LatinHypercube(d=d, rng=rng)
# s = xlim[0] + lh_sampler.random(n=s_dim)*(xlim[1]-xlim[0])
# # sort s
# # s = np.sort(s, axis=0)
# s = jnp.array(s)
# # s = jnp.array([-2.5, 2.5]).reshape(-1, 1)

# Sobol
sobol_sampler = qmc.Sobol(d=3, scramble=True, seed=0)
s = sobol_sampler.random_base2(m=int(np.log2(ns)))
s[:,0] = xlim[0] + s[:,0]*(xlim[1]-xlim[0])
# s[:,0] = vels[-1]
s[:,1] = ylim[0] + s[:,1]*(ylim[1]-ylim[0])
s[:,2] = zlim[0] + s[:,2]*(zlim[1]-zlim[0])
s = jnp.array(s)
# s = jnp.array([-2.5, 2.5]).reshape(-1, 1)

grad_noise_var = 5e-3
grad_noise_matrix = jnp.eye(3*len(s)) * grad_noise_var
grad_noise_matrix = np.asarray(grad_noise_matrix)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
# fig, ax = plot_3d_slices(f0u_exact, (u[:,0], u[:,1], u[:,2]), z_values=[1.0], vmin=vmin, vmax=vmax,xlim=xlim,ylim=ylim,zlim=zlim, figsize=fig_size)
ax.scatter(t[:,0], t[:,1], t[:,2], c='black', label='Data Points', alpha=0.5, marker='x')
ax.scatter(s[:,0], s[:,1], s[:,2], c='tab:orange', label='Virtual Points', alpha=0.5)
ax.set_xlabel("Velocity")
ax.set_ylabel("Time")
ax.set_zlabel("Space")
ax.set_box_aspect(aspect=None, zoom=0.85)
plt.tight_layout()

if save_fig:
    plt.savefig(file_name + "/virtual_points.pdf", bbox_inches="tight")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
# ax.scatter(t[:,0], t[:,1], t[:,2], c='black', label='Data Points', alpha=0.7)
ax.scatter(vels[vel_indices], ts[t_indices], xs[x_indices], c=ys[vel_indices, t_indices, x_indices], vmin=vmin, vmax=vmax, marker="x", alpha=1)
ax.scatter(s[:,0], s[:,1], s[:,2], c='tab:orange', label='Virtual Points', alpha=0.5)
ax.set_xlabel("Velocity")
ax.set_ylabel("Time")
ax.set_zlabel("Space")
ax.set_box_aspect(aspect=None, zoom=0.85)
plt.tight_layout()

if save_fig:
    plt.savefig(file_name + "/virtual_points_2.pdf", bbox_inches="tight")


## Prepare matrices

In [ ]:
kernel_xy = RBF(lengthscale=l, variance=sigma_square)
kernel_dx1y = RBFDX1Y(lengthscale=l, variance=sigma_square)
kernel_xdy1 = RBFXDY1(lengthscale=l, variance=sigma_square)
kernel_dx1dy1 = RBFDX1DY1(lengthscale=l, variance=sigma_square)

# added
kernel_dx2y = RBFDX2Y(lengthscale=l, variance=sigma_square)
kernel_xdy2 = RBFXDY2(lengthscale=l, variance=sigma_square)
kernel_dx2dy1 = RBFDX2DY1(lengthscale=l, variance=sigma_square)
kernel_dx1dy2 = RBFDX1DY2(lengthscale=l, variance=sigma_square)
kernel_dx2dy2 = RBFDX2DY2(lengthscale=l, variance=sigma_square)

# second added
kernel_dx3y = RBFDX3Y(lengthscale=l, variance=sigma_square)
kernel_xdy3 = RBFXDY3(lengthscale=l, variance=sigma_square)
kernel_dx3dy1 = RBFDX3DY1(lengthscale=l, variance=sigma_square)
kernel_dx3dy2 = RBFDX3DY2(lengthscale=l, variance=sigma_square)
kernel_dx3dy3 = RBFDX3DY3(lengthscale=l, variance=sigma_square)
kernel_dx1dy3 = RBFDX1DY3(lengthscale=l, variance=sigma_square)
kernel_dx2dy3 = RBFDX2DY3(lengthscale=l, variance=sigma_square)

K00tt = kernel_xy.gram(t).to_dense()
K10st = kernel_dx1y.cross_covariance(s, t)
K01ts = kernel_xdy1.cross_covariance(t, s)
K11ss = kernel_dx1dy1.gram(s).to_dense()

# added
K01ts2 = kernel_xdy2.cross_covariance(t, s)
K10st2 = kernel_dx2y.cross_covariance(s, t)
K11ss21 = kernel_dx2dy1.cross_covariance(s, s)
K11ss12 = kernel_dx1dy2.cross_covariance(s, s)
K11ss22 = kernel_dx2dy2.gram(s).to_dense()

# second added
K10st3 = kernel_dx3y.cross_covariance(s, t)
K01ts3 = kernel_xdy3.cross_covariance(t, s)
K11ss31 = kernel_dx3dy1.cross_covariance(s, s)
K11ss32 = kernel_dx3dy2.cross_covariance(s, s)
K11ss33 = kernel_dx3dy3.gram(s).to_dense()
k11ss13 = kernel_dx1dy3.cross_covariance(s, s)
k11ss23 = kernel_dx2dy3.cross_covariance(s, s)

f0t_prior_mean = opt_posterior.prior.mean_function(t)
f1s_prior_mean = jnp.zeros(3*s_dim)
f0u_prior_mean = opt_posterior.prior.mean_function(u).flatten()

# added
# f1s_prior_mean2 = jnp.zeros(s_dim)

K00tt = np.asarray(K00tt)
K10st = np.asarray(K10st)
K01ts = np.asarray(K01ts)
K11ss = np.asarray(K11ss)

# added
K01ts2 = np.asarray(K01ts2)
K10st2 = np.asarray(K10st2)
K11ss21 = np.asarray(K11ss21)
K11ss12 = np.asarray(K11ss12)
K11ss22 = np.asarray(K11ss22)

# seoncd added
K10st3 = np.asarray(K10st3)
K01ts3 = np.asarray(K01ts3)
K11ss31 = np.asarray(K11ss31)
K11ss32 = np.asarray(K11ss32)
K11ss33 = np.asarray(K11ss33)
k11ss13 = np.asarray(k11ss13)
k11ss23 = np.asarray(k11ss23)

K00tu = np.asarray(kernel_xy.cross_covariance(t, u))
K10su = np.asarray(kernel_dx1y.cross_covariance(s, u))
K10su2 = np.asarray(kernel_dx2y.cross_covariance(s, u))
K10su3 = np.asarray(kernel_dx3y.cross_covariance(s, u))
# stack K10su and K10su2 vertically
K10su = np.vstack((K10su, K10su2, K10su3))
K00uu = np.asarray(kernel_xy.gram(u).to_dense())

In [ ]:
# stack K01ts and K01ts2 horizontally
K01ts = np.hstack((K01ts, K01ts2, K01ts3))
# stack K10st and K10st2 vertically
K10st = np.vstack((K10st, K10st2, K10st3))
# stack sub-matrices vertically
K11ss_left = np.vstack((K11ss, K11ss21, K11ss31))
K11ss_middle = np.vstack((K11ss12, K11ss22, K11ss32))
K11ss_right = np.vstack((k11ss13, k11ss23, K11ss33))
# stack horizontally
K11ss = np.hstack((K11ss_left, K11ss_middle, K11ss_right))

In [ ]:
no_of_warmups = 1000
no_of_samples = 10000

## Truncated NUTS

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
start_time = timeit.default_timer()
original_samples, samples = nuts_truncated_sample_torch(K01ts, K11ss, K10st, K00tt, fun_noise_matrix, grad_noise_matrix, f0t_prior_mean, f0t, no_of_warmups, no_of_samples)
end_time = timeit.default_timer()
time_elapsed = end_time - start_time

samples.plot_ci()

if save_samples:
    np.savez(file_name + "/truncated_nuts_samples.npz", samples=original_samples.samples, ess=original_samples.compute_ess(), time=time_elapsed)

In [ ]:
np.random.seed(123)
f0u_samples = draw_samples_with_derivative_enhanced_gp(K00tt, K01ts, K10st, K11ss, K00tu, K10su, K00uu, fun_noise_var, grad_noise_var, f0t_prior_mean, f1s_prior_mean, f0u_prior_mean, f0t, samples.samples)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
im = ax.scatter(u[:,0], u[:,1], u[:,2], c=f0u_samples.mean(), vmin=vmin, vmax=vmax)
ax.set_xlabel("V")
ax.set_ylabel("T")
ax.set_zlabel("X")
# plt.colorbar(im)

info_string = "%.2e %.2e %.2e %.2e" % (mean_squared_error(f0u_exact, f0u_samples.mean()), (f0u_samples.compute_ci()[1]-f0u_samples.compute_ci()[0]).mean(), no_of_samples/original_samples.compute_ess().min(), samples.compute_ess().min()/time_elapsed)
info_strings.append(info_string)
print(info_string)


In [ ]:
fig, ax = plot_3d_slices(f0u_samples.mean(), (u[:,0], u[:,1], u[:,2]), z_values=[1.0], vmin=vmin, vmax=vmax,xlim=xlim,ylim=ylim,zlim=zlim, figsize=fig_size)
# ax.scatter(t[:,0], t[:,1], t[:,2], c='black', marker='x', alpha=0.7)
# ax.scatter(s[:,0], s[:,1], s[:,2], c="tab:orange", marker='o', alpha=0.7)
ax.set_xlabel("Velocity")
ax.set_ylabel("Time")
ax.set_zlabel("Space")
plt.tight_layout()
plt.savefig(file_name + "/truncated_nuts_mean.pdf", bbox_inches="tight")

fig, ax = plot_3d_slices(f0u_samples.compute_ci()[1]-f0u_samples.compute_ci()[0], (u[:,0], u[:,1], u[:,2]), z_values=[1.0], vmin=std_colorbar_min, vmax= std_colorbar_max,xlim=xlim,ylim=ylim,zlim=zlim, figsize=fig_size)
# ax.scatter(t[:,0], t[:,1], t[:,2], c='black', marker='x', alpha=0.7)
# ax.scatter(s[:,0], s[:,1], s[:,2], c="tab:orange", marker='o', alpha=0.7)
ax.set_xlabel("Velocity")
ax.set_ylabel("Time")
ax.set_zlabel("Space")
plt.tight_layout()
plt.savefig(file_name + "/truncated_nuts_ci.pdf", bbox_inches="tight")



In [ ]:
# Example usage with f0u_samples.mean() data
fig, ax = plot_isosurfaces(
    coords=u,                     # Your 3D coordinates
    values=f0u_samples.mean(),    # Your data values
    # isovalues=np.linspace(0.05, 0.9, 7),        # Plot isosurfaces at these values
    isovalues = np.linspace(0.1, 0.9, 9),
    # isovalues = [0.8, 0.95],
    # colors=['tab:blue', 'tab:red', 'black', 'tab:green', 'tab:orange'],  # Colors for each isosurface
    alphas=0.5,  # Transparency for each isosurface
    show_data_points=False,
    data_points=t,                 # Optional: show training points
    figsize = (9, 9),
    grid_size = 10
)
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_zlim(zlim)
# Add a legend without the decimal_places parameter
plt.show()

# Save the figure if needed
fig.savefig(file_name + "/truncated_nuts.pdf", bbox_inches="tight")

## Nonlinear NUTS

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
start_time = timeit.default_timer()
original_samples, samples = nuts_nonlinear_sample_torch(K01ts, K11ss, K10st, K00tt, fun_noise_matrix, grad_noise_matrix, f0t_prior_mean, f0t, no_of_warmups, no_of_samples)
end_time = timeit.default_timer()
time_elapsed = end_time - start_time

samples.plot_ci()

if save_samples:
    np.savez(file_name + "/nonlinear_nuts_samples.npz", samples=original_samples.samples, ess=original_samples.compute_ess(), time=time_elapsed)

In [ ]:
np.random.seed(123)
f0u_samples = draw_samples_with_derivative_enhanced_gp(K00tt, K01ts, K10st, K11ss, K00tu, K10su, K00uu, fun_noise_var, grad_noise_var, f0t_prior_mean, f1s_prior_mean, f0u_prior_mean, f0t, samples.samples)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
im = ax.scatter(u[:,0], u[:,1], u[:,2], c=f0u_samples.mean(), vmin=vmin, vmax=vmax)
ax.set_xlabel("V")
ax.set_ylabel("T")
ax.set_zlabel("X")
# plt.colorbar(im)


info_string = "%.2e %.2e %.2e %.2e" % (mean_squared_error(f0u_exact, f0u_samples.mean()), (f0u_samples.compute_ci()[1]-f0u_samples.compute_ci()[0]).mean(), no_of_samples/original_samples.compute_ess().min(), samples.compute_ess().min()/time_elapsed)
info_strings.append(info_string)
print(info_string)

In [ ]:
fig, ax = plot_3d_slices(f0u_samples.mean(), (u[:,0], u[:,1], u[:,2]), z_values=[1.0], vmin=vmin, vmax=vmax,xlim=xlim,ylim=ylim,zlim=zlim, figsize=fig_size)
# ax.scatter(t[:,0], t[:,1], t[:,2], c='black', marker='x', alpha=0.7)
# ax.scatter(s[:,0], s[:,1], s[:,2], c="tab:orange", marker='o', alpha=0.7)
ax.set_xlabel("Velocity")
ax.set_ylabel("Time")
ax.set_zlabel("Space")
plt.tight_layout()
plt.savefig(file_name + "/nonlinear_nuts_mean.pdf", bbox_inches="tight")

fig, ax = plot_3d_slices(f0u_samples.compute_ci()[1]-f0u_samples.compute_ci()[0], (u[:,0], u[:,1], u[:,2]), z_values=[1.0], vmin=std_colorbar_min, vmax= std_colorbar_max,xlim=xlim,ylim=ylim,zlim=zlim, figsize=fig_size)
# ax.scatter(t[:,0], t[:,1], t[:,2], c='black', marker='x', alpha=0.7)
# ax.scatter(s[:,0], s[:,1], s[:,2], c="tab:orange", marker='o', alpha=0.7)
ax.set_xlabel("Velocity")
ax.set_ylabel("Time")
ax.set_zlabel("Space")
plt.tight_layout()
plt.savefig(file_name + "/nonlinear_nuts_ci.pdf", bbox_inches="tight")



In [ ]:
# Example usage with f0u_samples.mean() data
fig, ax = plot_isosurfaces(
    coords=u,                     # Your 3D coordinates
    values=f0u_samples.mean(),    # Your data values
    # isovalues=np.linspace(0.05, 0.9, 7),        # Plot isosurfaces at these values
    isovalues = np.linspace(0.1, 0.9, 9),
    # isovalues = [0.8, 0.95],
    # colors=['tab:blue', 'tab:red', 'black', 'tab:green', 'tab:orange'],  # Colors for each isosurface
    alphas=0.5,  # Transparency for each isosurface
    show_data_points=False,
    data_points=t,                 # Optional: show training points
    figsize = (9, 9),
    grid_size = 10
)
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_zlim(zlim)
# Add a legend without the decimal_places parameter
plt.show()

# Save the figure if needed
fig.savefig(file_name + "/nonlinear_nuts.pdf", bbox_inches="tight")

## RTO

In [ ]:
np.random.seed(0)
start_time = timeit.default_timer()
samples = rto_sample(K01ts, K11ss, K10st, K00tt, fun_noise_matrix, grad_noise_matrix, f0t_prior_mean, f0t, no_of_warmups, no_of_samples)
end_time = timeit.default_timer()
time_elapsed = end_time - start_time
plt.subplots()
samples.plot_ci()

if save_samples:
    np.savez(file_name + "/rto_samples.npz", samples=samples.samples, ess=samples.compute_ess(), time=time_elapsed)

In [ ]:
np.random.seed(123)
f0u_samples = draw_samples_with_derivative_enhanced_gp(K00tt, K01ts, K10st, K11ss, K00tu, K10su, K00uu, fun_noise_var, grad_noise_var, f0t_prior_mean, f1s_prior_mean, f0u_prior_mean, f0t, samples.samples)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
im = ax.scatter(u[:,0], u[:,1], u[:,2], c=f0u_samples.mean(), vmin=vmin, vmax=vmax)
# ax.scatter(t[:,0], t[:,1], t[:,2], c='gray', label='Data Points')
# ax.scatter(s[:,0], s[:,1], s[:,2], c='red', label='Virtual Points')
ax.set_xlabel("V")
ax.set_ylabel("T")
ax.set_zlabel("X")
# plt.colorbar(im)

info_string = "%.2e %.2e %.2e %.2e" % (mean_squared_error(f0u_exact, f0u_samples.mean()), (f0u_samples.compute_ci()[1]-f0u_samples.compute_ci()[0]).mean(), no_of_samples/samples.compute_ess().min(), samples.compute_ess().min()/time_elapsed)
info_strings.append(info_string)
print(info_string)

In [ ]:
fig, ax = plot_3d_slices(f0u_samples.mean(), (u[:,0], u[:,1], u[:,2]), z_values=[1.0], vmin=vmin, vmax=vmax,xlim=xlim,ylim=ylim,zlim=zlim, figsize=fig_size)
# ax.scatter(t[:,0], t[:,1], t[:,2], c='black', marker='x', alpha=0.7)
# ax.scatter(s[:,0], s[:,1], s[:,2], c="tab:orange", marker='o', alpha=0.7)
ax.set_xlabel("Velocity")
ax.set_ylabel("Time")
ax.set_zlabel("Space")
plt.tight_layout()
plt.savefig(file_name + "/rto_mean.pdf", bbox_inches="tight")

fig, ax = plot_3d_slices(f0u_samples.compute_ci()[1]-f0u_samples.compute_ci()[0], (u[:,0], u[:,1], u[:,2]), z_values=[1.0], vmin=std_colorbar_min, vmax= std_colorbar_max,xlim=xlim,ylim=ylim,zlim=zlim, figsize=fig_size)
# ax.scatter(t[:,0], t[:,1], t[:,2], c='black', marker='x', alpha=0.7)
# ax.scatter(s[:,0], s[:,1], s[:,2], c="tab:orange", marker='o', alpha=0.7)
ax.set_xlabel("Velocity")
ax.set_ylabel("Time")
ax.set_zlabel("Space")
plt.tight_layout()
plt.savefig(file_name + "/rto_ci.pdf", bbox_inches="tight")



In [ ]:
# Example usage with f0u_samples.mean() data
fig, ax = plot_isosurfaces(
    coords=u,                     # Your 3D coordinates
    values=f0u_samples.mean(),    # Your data values
    # isovalues=np.linspace(0.05, 0.9, 7),        # Plot isosurfaces at these values
    isovalues = np.linspace(0.1, 0.9, 9),
    # isovalues = [0.8, 0.95],
    # colors=['tab:blue', 'tab:red', 'black', 'tab:green', 'tab:orange'],  # Colors for each isosurface
    alphas=0.5,  # Transparency for each isosurface
    show_data_points=False,
    data_points=t,                 # Optional: show training points
    figsize = (9, 9),
    grid_size = 10
)
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_zlim(zlim)
# Add a legend without the decimal_places parameter
plt.show()

# Save the figure if needed
fig.savefig(file_name + "/rto.pdf", bbox_inches="tight")

## Summary

In [ ]:
for i in range(len(info_strings)):
    print(info_strings[i])

In [ ]:
# save info_strings to file
with open(file_name + "/info.txt", "w") as f:
    f.write(cpuinfo.get_cpu_info()["brand_raw"] + "\n")
    f.write("# MSE CI IAC ESS/S\n")
    for i in range(len(info_strings)):
        f.write(info_strings[i] + "\n")

In [ ]:
# Example usage with f0u_samples.mean() data
fig, ax = plot_isosurfaces(
    coords=u,                     # Your 3D coordinates
    values=f0u_exact.flatten(),    # Your data values
    # isovalues=np.linspace(0.05, 0.9, 7),        # Plot isosurfaces at these values
    isovalues = np.linspace(0.1, 0.9, 9),
    # colors=['tab:blue', 'tab:red', 'black', 'tab:green', 'tab:orange'],  # Colors for each isosurface
    alphas=0.5,  # Transparency for each isosurface
    show_data_points=False,
    data_points=t,                 # Optional: show training points
    figsize = (9, 9),
    grid_size = 10
)
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_zlim(zlim)
plt.show()

# Save the figure if needed
fig.savefig(file_name + "/groundtruth.pdf", bbox_inches="tight")

In [ ]:
# Example usage with f0u_samples.mean() data
fig, ax = plot_isosurfaces(
    coords=u,                     # Your 3D coordinates
    values=unconstrained_gp.mean,    # Your data values
    # isovalues=np.array([0.1, 0.3, 0.5, 0.7, 0.9]),        # Plot isosurfaces at these values
    isovalues = np.linspace(0.1, 0.9, 9),        # Plot isosurfaces at these values
    # colors=['tab:blue', 'tab:red', 'black', 'tab:green', 'tab:orange'],  # Colors for each isosurface
    alphas=0.5,  # Transparency for each isosurface
    show_data_points=False,
    data_points=t,                 # Optional: show training points
    figsize = (9, 9),
    grid_size = 10
)
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_zlim(zlim)
plt.show()

# Save the figure if needed
fig.savefig(file_name + "/unconstrained.pdf", bbox_inches="tight")